In [ ]:
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
from skimage import io
from sklearn.model_selection import train_test_split

In [ ]:
image_list = os.listdir('imgs/')
print(len(image_list))

In [ ]:
def standarize_image(img_array,resize_pixels=256): 
  img = Image.fromarray(img_array)
 
  cols, rows = img.size
  extra = (rows-cols)/2
 
  if (extra>0):  
    crop_box = (0,extra,cols,cols+extra)
  else:
    crop_box = (-extra,0,rows-extra,rows) 
 
  standarized_image = img.crop(crop_box).resize((resize_pixels,resize_pixels), Image.ANTIALIAS)
 
  standarized_image_vector = (np.asarray(standarized_image).flatten().astype(np.float32)-128)/128 
 
  standarized_image_tensor = (np.asarray(standarized_image))
 
  return standarized_image_tensor, standarized_image_vector, standarized_image

In [ ]:
x_all = []

image_list = os.listdir('imgs/')
print(image_list)
    
for image in image_list:
    img_array = io.imread('imgs/' + image)
 
    (img_tensor,_,img) = standarize_image(img_array,128)
    x_all.append(img_tensor)

# Convert the list to a 4D array 
x_all = np.array(x_all)

In [ ]:
np.save('MRI_preprocessed',x_all)

In [ ]:
x_train = np.load("MRI_preprocessed.npy",allow_pickle=True)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(x_train[0],cmap='gray')
print(x_train.shape)

In [ ]:
image_size = 128
channels = 3

#1-32, 2-64, 3-96, 4-128
generate_res = 4

preview_rows = 4
preview_cols = 7
preview_margin = 16

seed_size = 100


epochs = 100
batch_size = 32
buffer_size = 60000

In [ ]:
train_dataset = tensorflow.data.Dataset.from_tensor_slices(x_train).shuffle(buffer_size).batch(batch_size)

In [ ]:
def generator_model(seed, num_channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D(size=(generate_res, generate_res)))
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    return model

In [ ]:
def discriminator_model(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
def save_images(cnt,noise):
  image_array = np.full(( 
      preview_margin + (preview_rows * (image_size+preview_margin)), 
      preview_margin + (preview_cols * (image_size+preview_margin)), 3), 255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(preview_rows):
      for col in range(preview_cols):
        r = row * (image_size+16) + preview_margin
        c = col * (image_size+16) + preview_margin
        image_array[r:r+image_size,c:c+image_size] = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join("GAN images",'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [ ]:
generator = generator_model(seed_size, channels)

noise = tensorflow.random.normal([1, seed_size])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0,:,:,0])

In [ ]:
image_shape = (image_size, image_size, channels)

discriminator = discriminator_model(image_shape)
decision = discriminator(generated_image)
print(decision)

In [ ]:
cross_entropy = tensorflow.keras.losses.BinaryCrossentropy()

def discrim_loss(real, fake):
  real_loss = cross_entropy(tensorflow.ones_like(real), real)
  fake_loss = cross_entropy(tensorflow.zeros_like(fake), fake)
  total_loss = real_loss + fake_loss
  return total_loss

def gener_loss(fake):
  loss = cross_entropy(tensorflow.ones_like(fake), fake)
  return loss

In [ ]:
gener_opt = Adam(1.5e-4, 0.5)
discrim_opt = Adam(1.5e-4, 0.5)

In [ ]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [ ]:
def train_discriminator(real, fake):
    n = real.size(0)
    discrim_opt.zero_grad()

In [ ]:
def train(dataset, epochs):
  fixed_seed = np.random.normal(0,1, (preview_rows*preview_cols, seed_size))
  
  start = time.time()

  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list = []

    for image_batch in dataset:
      t = train_step(image_batch)
      gen_loss_list.append(t[0])
      disc_loss_list.append(t[1])

    g_loss = sum(gen_loss_list) / len(gen_loss_list)
    d_loss = sum(disc_loss_list) / len(disc_loss_list)

    epoch_elapsed = time.time() - epoch_start
    print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss}, {hms_string(epoch_elapsed)}')
    if epoch % 20 == 0:
      save_images(epoch, fixed_seed)